# Harnessing Weather Insights for Accurate Energy Load Forecasting

In [1]:
%pip install -r requirements.txt

Note: you may need to restart the kernel to use updated packages.


### Import important libraries

In [2]:
import platform
import findspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.window import Window
from pyspark.ml.linalg import DenseVector
from pyspark.ml.feature import StandardScaler, VectorAssembler
from pyspark.ml.regression import LinearRegression
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.ml.evaluation import RegressionEvaluator

from pathlib import Path

In [3]:
# Initialize via the full spark path
if platform.system() == 'Windows':
    print("Windows OS detected")
    findspark.init("C:/Spark/spark-3.5.4-bin-hadoop3") # For my local machine
else:
    findspark.init("/usr/local/spark/")

Windows OS detected


In [4]:
# Build the SparkSession
spark = SparkSession.builder \
    .master("local[*]") \
    .appName("Linear Regression Model") \
    .config("spark.executor.memory", "16g") \
    .config("spark.sql.shuffle.partitions", "8") \
    .config("spark.jars.packages", "com.databricks:spark-xml_2.12:0.15.0") \
    .config("spark.executor.heartbeatInterval", "100s") \
    .config("spark.sql.session.timeZone", "UTC") \
    .getOrCreate()

   
# Main entry point for Spark functionality. A SparkContext represents the
# connection to a Spark cluster, and can be used to create :class:`RDD` and
# broadcast variables on that cluster.      
sc = spark.sparkContext

### Preprocessing I: Read in Weather and Load Data

In [5]:
# Read in the data

# Folder Structure
# data
# |-- geosphere
# |   |-- YYYY
# |      |-- MM.csv
# |      |-- MM.csv
# |
# |-- transparency
# |   |-- YYYY
# |      |-- MM.xml
# |      |-- MM.xml

# Loop through the geosphere folder and read in the data

# Define the base data folder
base_path = Path("./data/geosphere")

# Collect all data frames first to optimize the union operation
dfs = []

for year_folder in base_path.iterdir():
    if year_folder.is_dir():
        for month_file in year_folder.glob("*.csv"):
            print(f"Reading in {month_file}")

            df = spark.read.csv(str(month_file), header=True, inferSchema=True)

            # Convert the time column (string) to a timestamp
            df = df.withColumn("time", to_timestamp(col("time"), "yyyy-MM-dd'T'HH:mmXXX"))
            
            dfs.append(df)
            
            print(f"Read in {df.count()} rows")

if dfs:
    # Combine all DataFrames
    weather = dfs[0]
    for df in dfs[1:]:
        weather = weather.union(df)

    # Aggregate measurements (average from different stations)
    weather = (
        weather.groupBy("time")
        .agg(
            avg("rr").alias("avg_rr"),
            avg("tl").alias("avg_tl"),
            avg("p").alias("avg_p"),
            avg("so_h").alias("avg_so_h"),
            avg("ff").alias("avg_ff"),
        )
        .orderBy("time")
    )

    # Log-transform the measurements (data for rainfall and windspeed is skewed)
    weather = weather.withColumn("log_rr", log1p(col("avg_rr")))
    weather = weather.withColumn("log_ff", log1p(col("avg_ff")))
    
    weather.drop("avg_rr", "avg_ff")
    
    weather.show(10, truncate=False)
    
    print(weather.count())
    weather.printSchema()
else:
    print("No data found")


Reading in data\geosphere\2015\01.csv
Read in 7500 rows
Reading in data\geosphere\2015\02.csv
Read in 8076 rows
Reading in data\geosphere\2015\03.csv
Read in 8940 rows
Reading in data\geosphere\2015\04.csv
Read in 8652 rows
Reading in data\geosphere\2015\05.csv
Read in 8940 rows
Reading in data\geosphere\2015\06.csv
Read in 8652 rows
Reading in data\geosphere\2015\07.csv
Read in 8940 rows
Reading in data\geosphere\2015\08.csv
Read in 8940 rows
Reading in data\geosphere\2015\09.csv
Read in 8652 rows
Reading in data\geosphere\2015\10.csv
Read in 8940 rows
Reading in data\geosphere\2015\11.csv
Read in 8652 rows
Reading in data\geosphere\2015\12.csv
Read in 8940 rows
Reading in data\geosphere\2016\01.csv
Read in 8940 rows
Reading in data\geosphere\2016\02.csv
Read in 8364 rows
Reading in data\geosphere\2016\03.csv
Read in 8940 rows
Reading in data\geosphere\2016\04.csv
Read in 8652 rows
Reading in data\geosphere\2016\05.csv
Read in 8940 rows
Reading in data\geosphere\2016\06.csv
Read in 86

In [6]:
# Loop through the transparency folder and read in the energy data

# Define base path for transparency data
base_path = Path("./data/transparency")

# Collect DataFrames before performing union (optimization)
dfs = []

for year_folder in base_path.iterdir():
    if year_folder.is_dir():
        for month_file in year_folder.glob("*.xml"):
            print(f"Reading transparency data: {month_file}")

            # Read XML data
            df = spark.read.format('xml').option("rowTag", "GL_MarketDocument").load(str(month_file))

            # Extract and explode relevant fields
            df_filtered = df.select(
                col("TimeSeries.Period.timeInterval.start").alias("start_time"),
                col("TimeSeries.Period.timeInterval.end").alias("end_time"),
                col("TimeSeries.Period.resolution").alias("resolution"),
                explode(col("TimeSeries.Period.Point")).alias("Point")  # Flatten Points
            ).select(
                col("start_time"),
                col("end_time"),
                col("resolution"),
                col("Point.position").cast("int").alias("position"),
                col("Point.quantity").cast("double").alias("quantity")
            )

            # Convert ISO 8601 duration (e.g., "PT15M") to minutes dynamically
            df_fixed = df_filtered.withColumn(
                "interval_minutes",
                expr("CAST(SUBSTRING(resolution, 3, LENGTH(resolution) - 3) AS INT)")  # Extracts "15" from "PT15M"
            ).withColumn(
                "actual_time",
                expr("start_time + (position - 1) * interval_minutes * interval 1 minute")
            ).select(
                col("actual_time"),
                col("quantity")
            )
            
            # Aggregate to hourly intervals
            df_hourly = df_fixed.withColumn(
                "hourly_time", date_trunc("hour", col("actual_time"))  # Round down to the hour
            ).groupBy("hourly_time").agg(
                sum("quantity").alias("hourly_quantity")  # Sum all sub-hourly measurements
            )

            # Rename to match the structure of other time series
            df_hourly = df_hourly.select(
                col("hourly_time").alias("timestamp"), col("hourly_quantity").alias("quantity")
            ).orderBy("timestamp")
            
            print(f"Read in {df_hourly.count()} values")
            
            # Append DataFrame to list
            dfs.append(df_hourly)

# Merge all collected DataFrames
if dfs:
    Load = dfs[0]
    for df in dfs[1:]:
        Load = Load.union(df)

    Load.show(10)
    Load.printSchema()
else:
    print("No data found.")



Reading transparency data: data\transparency\2015\01.xml
Read in 624 values
Reading transparency data: data\transparency\2015\02.xml
Read in 672 values
Reading transparency data: data\transparency\2015\03.xml
Read in 744 values
Reading transparency data: data\transparency\2015\04.xml
Read in 720 values
Reading transparency data: data\transparency\2015\05.xml
Read in 744 values
Reading transparency data: data\transparency\2015\06.xml
Read in 720 values
Reading transparency data: data\transparency\2015\07.xml
Read in 744 values
Reading transparency data: data\transparency\2015\08.xml
Read in 744 values
Reading transparency data: data\transparency\2015\09.xml
Read in 720 values
Reading transparency data: data\transparency\2015\10.xml
Read in 744 values
Reading transparency data: data\transparency\2015\11.xml
Read in 720 values
Reading transparency data: data\transparency\2015\12.xml
Read in 744 values
Reading transparency data: data\transparency\2016\01.xml
Read in 744 values
Reading tran

### Preprocessing II: Combine both Data Frames

In [7]:
if Load is not None and weather is not None:
    # Join the data into a single DataFrame
    data = Load.join(weather, Load.timestamp == weather.time, "inner").drop("time")
    
    # Rename columns for better understanding
    data = data.withColumnRenamed("timestamp", "time")
    data = data.withColumnRenamed("quantity", "load")

    data = data.withColumnRenamed("log_rr", "rainfall")
    data = data.withColumnRenamed("avg_tl", "temperature")
    data = data.withColumnRenamed("avg_p", "pressure")
    data = data.withColumnRenamed("avg_so_h", "sunshine_duration")
    data = data.withColumnRenamed("log_ff", "wind_speed")
    
    # Add Lag features
    
    # Define window for lagging (ordered by time)
    window_spec = Window().partitionBy().orderBy("time")

    # Add lag features (previous hour's values)
    data = data.withColumn("rainfall_lag_1", lag("rainfall", 1).over(window_spec))
    data = data.withColumn("wind_speed_lag_1", lag("wind_speed", 1).over(window_spec))
    
    # Add lag features (previous day's values)
    data = data.withColumn("load_lag_6h_prev_day", lag("load", 6).over(window_spec))
    data = data.withColumn("load_lag_12h_prev_day", lag("load", 12).over(window_spec))
    data = data.withColumn("load_lag_18h_prev_day", lag("load", 18).over(window_spec))
    data = data.withColumn("load_lag_24h_prev_day", lag("load", 24).over(window_spec))
    
    # Drop rows with missing values
    data = data.dropna()
    
    # Reorder the columns
    data = data.select("time",
                        "load", 
                        "rainfall", 
                        "rainfall_lag_1",
                        "temperature", 
                        "pressure", 
                        "sunshine_duration", 
                        "wind_speed", 
                        "wind_speed_lag_1", 
                        "load_lag_6h_prev_day",
                        "load_lag_12h_prev_day", 
                        "load_lag_18h_prev_day",
                        "load_lag_24h_prev_day")
    
    # Print the schema and stats
    data.describe().show()
    data.show(10)
    data.printSchema()

+-------+-----------------+-------------------+-------------------+-------------------+-----------------+-------------------+------------------+-------------------+--------------------+---------------------+---------------------+---------------------+
|summary|             load|           rainfall|     rainfall_lag_1|        temperature|         pressure|  sunshine_duration|        wind_speed|   wind_speed_lag_1|load_lag_6h_prev_day|load_lag_12h_prev_day|load_lag_18h_prev_day|load_lag_24h_prev_day|
+-------+-----------------+-------------------+-------------------+-------------------+-----------------+-------------------+------------------+-------------------+--------------------+---------------------+---------------------+---------------------+
|  count|            87528|              87528|              87528|              87528|            87528|              87528|             87528|              87528|               87528|                87528|                87528|               

### Machine Learning

In [ ]:
# Keep original timestamp before converting to Unix seconds
unix_time_data = data.withColumn("unix_time", unix_timestamp("time"))  # Store Unix time separately

# Extract day of the year (1-365/366) from the original timestamp column
unix_time_data = unix_time_data.withColumn("day_of_year", dayofyear(col("time")))

# Extract time of day in seconds (seconds since midnight)
unix_time_data = unix_time_data.withColumn(
    "time_of_day", expr("hour(time) * 3600 + minute(time) * 60 + second(time)")
)

# Normalize day of the year to range (0, 2π) for periodic encoding
ml_data = unix_time_data.withColumn("day_cos", cos(2 * 3.1416 * col("day_of_year") / 365))

# Normalize time of day
ml_data = ml_data.withColumn("time_cos", cos(2 * 3.1416 * col("time_of_day") / 86400))

# Define new feature columns
#feature_columns = ["day_cos", "time_cos", "is_weekend", "rainfall", "temperature", "pressure", "sunshine_duration", "wind_speed"]

feature_columns = [     "day_cos",
                        "time_cos",
                        "rainfall", 
                        "rainfall_lag_1",
                        "temperature", 
                        "pressure", 
                        "sunshine_duration", 
                        "wind_speed", 
                        "wind_speed_lag_1", 
                        "load_lag_6h_prev_day",
                        "load_lag_12h_prev_day", 
                        "load_lag_18h_prev_day",
                        "load_lag_24h_prev_day"
                        ]


assembler = VectorAssembler(inputCols=feature_columns, outputCol="features")


# Transform data
df_ml = assembler.transform(ml_data).select("load", "features")

df_ml.cache()

# Show transformed data
df_ml.show(5, truncate=False)



+-------+---------------------------------------------------------------------------------------------------------------------------------------------------------------+
|load   |features                                                                                                                                                       |
+-------+---------------------------------------------------------------------------------------------------------------------------------------------------------------+
|22272.0|[0.9927486885854637,1.0,0.0,0.0,-3.2583333333333333,981.5,0.0,0.5643663906940205,0.5831462853456167,26701.0,24814.0,21746.0,20881.0]                           |
|22567.0|[0.9927486885854637,0.9659256678396477,0.0,0.0,-3.391666666666667,981.7727272727275,0.0,0.5158131652770297,0.5643663906940205,24159.0,23975.0,23284.0,20059.0] |
|22190.0|[0.9927486885854637,0.8660247915829389,0.0,0.0,-3.3250000000000006,982.1454545454545,0.0,0.6286086594223741,0.5158131652770297,22928.0,24038.

In [10]:
scaler = StandardScaler(inputCol="features", outputCol="features_scaled")
scaled_df = scaler.fit(df_ml).transform(df_ml).select("load", "features_scaled")
scaled_df = scaled_df.withColumnRenamed("features_scaled", "features")
scaled_df.cache()

scaled_df.show(5, truncate=False)

+-------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|load   |features                                                                                                                                                                                                 |
+-------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|22272.0|[1.4045177045504165,1.4142038681594762,0.0,0.0,-0.38818276818347563,127.75820620739627,0.0,2.029935382893637,2.097461109233491,4.898129918520666,4.551971665797769,3.9891372592197603,3.8302297670842345]|
|22567.0|[1.4045177045504165,1.366015815813355,0.0,0.0,-0.4040674850400885,127.79370610260513,0.0,1.8552972190115569,2.029913566434895,4.431816063126503

In [11]:
# Randomly splits this :class:`DataFrame` with the provided weights.
train_data, test_data = scaled_df.randomSplit([.8,.2],seed=1234)

train_data.take(5)

[Row(load=2653.0, features=DenseVector([1.2055, 0.0, 0.4016, 1.089, 0.5897, 126.8104, 0.0, 4.8488, 4.6239, 1.0748, 1.9203, 1.919, 5.6502])),
 Row(load=3793.0, features=DenseVector([1.2055, 1.2247, 0.0461, 0.0461, 0.7436, 126.7062, 0.0, 4.841, 4.7381, 0.7459, 1.0594, 2.0441, 4.8985])),
 Row(load=3841.0, features=DenseVector([1.2055, 0.7071, 0.1371, 0.3148, 0.5927, 126.782, 0.0, 4.6073, 4.7461, 0.991, 1.4364, 1.9603, 5.0809])),
 Row(load=4467.0, features=DenseVector([1.2055, -1.366, 0.5293, 0.6128, 0.6592, 126.7121, 0.0775, 4.193, 4.6814, 1.7728, 2.0922, 5.3615, 5.5706])),
 Row(load=4573.0, features=DenseVector([1.2055, 0.366, 0.3148, 0.4016, 0.5987, 126.7997, 0.0, 4.7461, 4.8488, 0.8194, 1.7728, 2.0922, 5.3611]))]

In [12]:
# Linear regression.
# The learning objective is to minimize the specified loss function, with regularization.
#More about regParam and elasticNetParam (seen as penalties on the loss function that minimises error in predicitons) can be found here: https://runawayhorse001.github.io/LearningApacheSpark/reg.html
# The maxIter parameter sets an upper limit on the number of iterations the optimization algorithm can perform regarding the loss function
# By default the LinearRegression function expects another column named "features"

lr = LinearRegression(labelCol="load", maxIter=500)

# Fits a model to the input dataset with optional parameters.
linearModel = lr.fit(train_data)

# Get the summary of the model
trainingSummary = linearModel.summary

# Print the Score of the model
print("RMSE: %f" % trainingSummary.rootMeanSquaredError)
print("r2: %f" % trainingSummary.r2)

RMSE: 2643.196536
r2: 0.765227


In [ ]:
# Define the model
lr = LinearRegression(featuresCol="features", labelCol="load")

# Define hyperparameter grid
paramGrid = (ParamGridBuilder()
             .addGrid(lr.regParam, [0.001, 0.01, 0.1, 1.0])
             .addGrid(lr.elasticNetParam, [0.0, 0.5, 1.0])  # 0 = Ridge, 1 = Lasso
             .build())

# Define evaluator
evaluator = RegressionEvaluator(labelCol="load", predictionCol="prediction", metricName="rmse")

# Define cross-validation
cv = CrossValidator(estimator=lr, estimatorParamMaps=paramGrid, evaluator=evaluator, numFolds=5)

# Fit the model with cross-validation
cv_model = cv.fit(train_data)

# Get the best model
best_model = cv_model.bestModel
print(f"Best regParam: {best_model._java_obj.getRegParam()}")
print(f"Best elasticNetParam: {best_model._java_obj.getElasticNetParam()}")

# Make predictions
predictions = best_model.transform(test_data)

# Evaluate performance
rmse = evaluator.evaluate(predictions)
print(f"RMSE: {rmse}")

# Show predictions
predictions.select("load", "prediction").show(10, truncate=False)

r2 = evaluator.setMetricName("r2").evaluate(predictions)
print(f"R2: {r2}")

In [ ]:
import pandas as pd

# Get coefficients
coefficients = best_model.coefficients.toArray()

# Create a DataFrame for feature importance
coef_df = pd.DataFrame({"Feature": feature_columns, "Coefficient": coefficients})

# Sort by absolute coefficient values (fixing the issue)
coef_df = coef_df.reindex(coef_df["Coefficient"].abs().sort_values(ascending=False).index)

# Display feature importance
print("\nFeature Importance (Lasso Regression Coefficients):")
print(coef_df)

# Identify removed features (Lasso sets them to exactly 0)
removed_features = coef_df[coef_df["Coefficient"] == 0]["Feature"].tolist()
print(f"\nFeatures removed by Lasso: {removed_features}")



In [ ]:
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml.evaluation import RegressionEvaluator

# Define the Random Forest model
rf = RandomForestRegressor(featuresCol="features", labelCol="load", numTrees=100, maxDepth=10, seed=42)

# Train the Random Forest model
rf_model = rf.fit(train_data)

# Make predictions
rf_predictions = rf_model.transform(test_data)

# Evaluate performance
evaluator = RegressionEvaluator(labelCol="load", predictionCol="prediction", metricName="rmse")
rf_rmse = evaluator.evaluate(rf_predictions)

print(f"Random Forest RMSE: {rf_rmse}")

# Show predictions
rf_predictions.select("load", "prediction").show(10, truncate=False)

# Feature importance
rf_feature_importance = rf_model.featureImportances
print("\nFeature Importances (Random Forest):")
for i, imp in enumerate(rf_feature_importance):
    print(f"{feature_columns[i]}: {imp}")
